In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Setting the Order of Subsystems in a Group

By default, subsystems are executed in the same order in which they were added to
their parent Group.  In order to change this order, use the `set_order` method.

```{eval-rst}
    .. automethod:: openmdao.core.group.Group.set_order
        :noindex:
```

The list of names provided to `set_order` must contain every subsystem that has
been added to the Group.

```{Note}
Use caution when setting the order of execution of your subsystems, whether
by just calling `add_subsystem` in a specific order, or by later changing
the order using `set_order`.  If you choose an order that doesn't follow
the natural data flow order of your subsystems, your model may take longer
to converge.
```

## Usage

Change the execution order of components *C1* and *C3*.

In [ ]:
import openmdao.api as om


class ReportOrderComp(om.ExplicitComponent):
    """Adds name to list."""

    def __init__(self, order_list):
        super().__init__()
        self._order_list = order_list

    def compute(self, inputs, outputs):
        self._order_list.append(self.pathname)

# this list will record the execution order of our C1, C2, and C3 components
order_list = []

prob = om.Problem()
model = prob.model

model.add_subsystem('C1', ReportOrderComp(order_list))
model.add_subsystem('C2', ReportOrderComp(order_list))
model.add_subsystem('C3', ReportOrderComp(order_list))

prob.setup()
prob.run_model()

print(order_list)

In [ ]:
assert(order_list == ['C1', 'C2', 'C3'])

In [ ]:
# reset the shared order list
order_list[:] = []

prob.setup()
# now swap C2 and C1 in the order
model.set_order(['C2', 'C1', 'C3'])

# after changing the order, we must call setup again
prob.setup()
prob.run_model()

print(order_list)

In [ ]:
assert(order_list == ['C2', 'C1', 'C3'])